In [1]:
!pip install tweet-preprocessor
!pip install Sastrawi
import nltk
!nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 6.1 MB/s eta 0:00:00
/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import preprocessor as p
import re
import nltk
import string
import json

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf

from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Dense, LSTM
from tensorflow.keras.losses import CategoricalCrossentropy

# Load Model CNN + LSTM

In [4]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Model/cnnmodel')
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 64)            1280064   
                                                                 
 conv1d (Conv1D)             (None, 48, 128)           24704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 128)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               91600     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                        

In [5]:
from ipywidgets import widgets
from IPython.display import display
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing import sequence
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# input words
text = widgets.Text(
    value='',
    placeholder='Type Here!'
)
button = widgets.Button(description = 'Klasifikasi')
title = widgets.HTML('Form Klasifikasi kata')

# for render 
display(title)
display(text)
display(button)

# ==================setup for stem text===============================
# setup stopwords
stopwords = set(stopwords.words('indonesian'))
additional_stopwords = ['g', 'yg', 'tdk', 'gak', 'x', 'nya', 'rt', 'no', 'kk', 'sm', 'tp', 'pls', 'cb', 'sgt', 'jg', 'y']
new_stopwords        = stopwords.union(additional_stopwords)
# end

# setup stemer kata
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# end

def caseFolding(text):
    text = str(text).lower()
    text = ' '.join(re.sub("[@#][A-Za-z0-9_]+|([^0-9A-Za-z \t])|(\w+:\/\/\S+)+|<.*?>|^https?:\/\/.*[\r\n]*"," ", text).split(' '))
    text = re.sub(r"\d+", "", text)
    return text

def tokenizing(text):
    text = nltk.tokenize.word_tokenize(text)
    return text

def filtering(token):
    filter_ = [t for t in token if t not in new_stopwords]
    return " ".join(filter_)

def stemming(filtered_token):
    stemmed = stemmer.stem(filtered_token)
    return stemmed
# end setup===============================================================

# proses klasifikasi kalimat 
def on_button_clicked(b):
    #proses seleksi kalimat
    x = text.value
    _casefolding = caseFolding(x)
    _tokenizing  = tokenizing(_casefolding)
    _filtering   = filtering(_tokenizing)
    _stemming    = stemming(_filtering)
    kalimat      = _stemming
    input        = [kalimat]
    #end seleksi
    with open('/content/drive/MyDrive/Model/tokenizer.json') as json_file:
        json_string = json.load(json_file)
    tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string)
    # vectorizer = TfidfVectorizer(norm='l2') # Membuat TF ID-F Vectorizer
    # response   = vectorizer.fit_transform(input) # mendapatkan kata dari data train
    # feature_names  = vectorizer.get_feature_names_out()
    label = ['NETRAL', 'PORNOGRAFI', 'UMPATAN']
    
    #LSTM
    # teks Tokenizer
    num_words = 20000
    maxlen = 50

    # tokenizer = Tokenizer(num_words=num_words,oov_token="unk") ## unk = unknown
    # tokenizer.fit_on_texts(input)
    sequences_test = tokenizer1.texts_to_sequences(input)
    X_test = pad_sequences(sequences_test, padding='post', maxlen=maxlen)
    
    #cnn
    predictions = new_model.predict(X_test)
    predict_results = predictions.argmax(axis=1)
    label = ''
    if(predict_results == 0):
        label = "netral"
    elif(predict_results == 1):
        label = "pornografi"
    else:
        label = "umpatan"
    
    
    print(input)
    print("kalimat Awal: {}" .format(x))
    # print("kalimat stem= {}" .format(feature_names))
    # print("hasil tf-idf= {}" .format(response))
    print("hasil sequence= {}" .format(X_test))
    print(predictions)
    print(label)
    
button.on_click(on_button_clicked)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


HTML(value='Form Klasifikasi kata')

Text(value='', placeholder='Type Here!')

Button(description='Klasifikasi', style=ButtonStyle())

1/1 [==============================] - 1s 802ms/step
['halo selamat pagi']
kalimat Awal: halo selamat pagi
hasil sequence= [[1240  146  137    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]
[[0.992877   0.05434675 0.05100013]]
netral
1/1 [==============================] - 0s 22ms/step
['babi makan bisa']
kalimat Awal: Babi! Makan terus saja bisamu!
hasil sequence= [[  51   64 3018    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]
[[0.99294156 0.05418513 0.05060371]]
netral
1/1 [==============================] - 0s 21ms/step
['laku kaya jancuk']
kalimat Awal: kelakuan dia kaya jancuk
hasil sequence= [[  84  